# Bank marketing use case |  Monkey Patch your machine learning pipeline

## 0. Setup

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
from utils import data_prep, model_performance, create_observability_report

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
jan = pd.read_csv('../data/predict/jan-data.csv')
model = pickle.load(open('../models/model_log.cav','rb'))

## 1. Introduction

You are now aware of the potential datastrophes and failures that may happen in your model pipelines in Production. As a data scientist wanting to have more confidence in those pipelines, you have decided to prevent those failures by logging the events of your scripts. 

In this notebook, we will see how a logging library can be included upstream (from the creation of the dataset to the predict of the model) to avoid any `datastrophe` to happen. 

## 2. The data extraction script

The logging strategy wants to prevent any failure when you run the model. To be complete, you need to trace and monitor your dataflow as soon as possible. In the following cell, take a moment to acquaint yourself with the code we will run and log in the exercise. 

In [ ]:
customers_info = pd.read_csv('../data/predict/may-customers-data.csv')
contact_info = pd.read_csv('../data/predict/may-contact-data.csv')
business_info = pd.read_csv('../data/predict/may-business-data.csv')

customer360 = customers_info.merge(contact_info,on='id')

may = pd.merge(customer360,business_info)

may.to_csv('../data/predict/may-data.csv',index=False)

## 3. How to log the events of the script?

In this notebook, we will show you how you can simply log all events in the notebook in order to track the lineage and enable AI observability. 

We will use an open source library provided by Kensu. The library will monkey patch the common python libraries such as pandas and sklearn in order to keep records of what happens in the script. With this library, we will be able to extract the following information from the notebook:
- the data sources used and their schemas
- the lineages (also called data flow or data map) among those data sources
- the statistics and metrics of the data sources in that context 

To import the library, you need to add the kensu prefix to the library we need to monkeypatch:

$$\text{pandas} \rightarrow  \text{kensu.pandas}$$

Once imported, you can initialize the client with the `KensuProvider` object. Several parameters are available and you can find the list in the user documentation. 

The `Context` of the application is defined by its `process_name` (an identifier for your application), `project_names`(where the application is running), and `environment`.

In [ ]:
from kensu.utils.kensu_provider import KensuProvider
kensu = KensuProvider().initKensu(process_name='Exercise 3',
                            user_name='Sammy', 
                            code_location='https://gitlab.example.com', 
                            init_context=True, 
                            project_names=['O-Reilly'], 
                            environment="Production",
                            report_to_file=True,
                            offline_file_name='log_pandas_example.log')


### Example of a wrapped function:
``` python
def wrap_pandas_get_dummies(method):
    def wrapper(*args, **kwargs):
        kensu = KensuProvider().instance()
        df_result = method(*args, **kwargs)
        df = args[0] # see get_dummies definition (first arg is `data`)

        orig_ds = kensu.extractors.extract_data_source(df)
        orig_sc = kensu.extractors.extract_schema(orig_ds, df)
        result_ds = kensu.extractors.extract_data_source(df_result)
        result_sc = kensu.extractors.extract_schema(result_ds, df_result)

        col_dest = [k.name for k in result_sc.pk.fields]
        col_orig = [k.name for k in orig_sc.pk.fields]
        prefix = kwargs['prefix'] if 'prefix' in kwargs else None
        prefix_sep = kwargs['prefix_sep'] if 'prefix_sep' in kwargs else "_"
        columns = kwargs['columns'] if 'columns' in kwargs else None

        for col in col_dest:
            if col in col_orig:
                kensu.add_dependencies_mapping(result_sc.to_guid(), col, orig_sc.to_guid(), col)
            else:
                origin_col = col.split(prefix_sep)[0]
                if prefix:
                    if isinstance(prefix,list):
                        index = prefix.index(origin_col)
                        origin_col = columns[index]
                kensu.add_dependencies_mapping(result_sc.to_guid(), col, orig_sc.to_guid(), origin_col)

        return df_result

    wrapper.__doc__ = method.__doc__
    return wrapper

get_dummies = wrap_pandas_get_dummies(pd.get_dummies)
```

**Exercise:** Integrate the library in the following code and inspect the result 

- *Note:* the client has been already initialized in the previous cell

In [ ]:
#Write your code here

import kensu.pandas as pd

customers_info = pd.read_csv('../data/predict/may-customers-data.csv')
contact_info = pd.read_csv('../data/predict/may-contact-data.csv')
business_info = pd.read_csv('../data/predict/may-business-data.csv')

customer360 = customers_info.merge(contact_info,on='id')
may = pd.merge(customer360,business_info)
may.to_csv('../data/predict/may-data.csv',index=False)

In [ ]:
may

The result of the piece of code you've run is available in the `log_pandas_example.log` file:

In [ ]:
!cat log_pandas_example.log

This file is not easy to understand by itself. Fortunately, the `create_observability_report` function from the utils module will help you to generate documentation based on the log file. 

The function takes 2 arguments, the first one is the `log file` path, and the second is the name of the document with a `.pdf` extention that you want to generate. 

In [ ]:
create_observability_report('log_pandas_example.log','Report-example.pdf')

## Well done!

A new observability report is now available in your working directory. Take a moment to explore it. What more would you like to log? 

In the next exercise, we will see how you can activate those reports in order to have SLA and SLO in your AI pipelines.  